In [61]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
import time


In [94]:
def Generate_Data(place = "Mumbai",Property_type = "Hostels",index=1):
    
    Dataset = pd.DataFrame(columns=['City',
                           'Property Type',
                           'Property Name',
                           'Contact Number',
                           'Ratings'])
    Dataset.to_csv(place+"_"+Property_type+".csv")
    
    place = place
    Property_type = Property_type
    index = index
    while True :
        print(index)
        if(index <=100):
            link = "https://www.justdial.com/"+place+"/"+Property_type+"/page-"+str(index) 
            print(link)
            next_disabled,Content = Scrap_Data(link,place,Property_type)
            #Content.to_csv(place+"_"+Property_type+"_"+str(index)+".csv")
            #Dataset.append(Content,ignore_index=True)
            Content.to_csv(place+"_"+Property_type+".csv",mode='a', header=False)
            if(next_disabled):
                break
            else:
                index += 1
        else:
            break
    return Dataset
    
    

In [95]:
def Scrap_Data(link,place,Property_type):
    
    #dataset = pd.DataFrame(columns=['City','Property Type','Contact Number','Rating'])
    
    City = []
    Property = []
    Contact_Number = []
    Rating = []
    Name = []
    
    Last_Page = False
    
    #print(Property_type)
    link = link
    
    dict_icon_number = {'icon-dc':'+',
                        'icon-fe':'(',
                        'icon-hg':')',
                        'icon-ba':'-',
                        'icon-acb':'0',
                        'icon-yz':'1',
                        'icon-wx':'2',
                        'icon-vu':'3',
                        'icon-ts':'4',
                        'icon-rq':'5',
                        'icon-po':'6',
                        'icon-nm':'7',
                        'icon-lk':'8',
                        'icon-ji':'9',}
    
    
    
    if(link):
        url = link
        driver = webdriver.Firefox()
        driver.get(url)
    
        SCROLL_PAUSE_TIME = 4

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        pg_div  = soup.find_all('div',class_='col-sm-5 col-xs-8 store-details sp-detail paddingR0')
        
        for container in pg_div:
            
            City.append(place)
            Property.append(Property_type)
            
            #Hostel Name
            hostel_name = container.h2.span.a['title']
            Name.append(hostel_name)
            #print("Hostel Name : "+hostel_name)
            
            #Ratings
            rating = container.find('span',class_="green-box")
            if(rating):
                Rating.append(rating.text)
            else:
                Rating.append("0")
            #print("Rating : "+rating.text)
            
            #Mobile Number
            sub_container_mob_num = container.find('p',class_='contact-info')
            if(sub_container_mob_num):
                sub_container_mob_num = sub_container_mob_num.span
            else:
                #print("Contact Number : None")
                #print("\n")
                #print("----------------------------")
                Contact_Number.append(None)
                continue
            
            if(sub_container_mob_num.a.find('b')):
                sub_container_mob_num = sub_container_mob_num.a.b
            else:
                sub_container_mob_num = sub_container_mob_num.a
            
            
            Digits = []
            mobile_number = ''
            if(sub_container_mob_num):
                span_container = sub_container_mob_num.find_all('span')
                for s in span_container:
                    Digits.append(dict_icon_number[s['class'][1]])
                #print("Contact Number : "+''.join(map(str, Digits)) )
                mobile_number = ''.join(map(str, Digits))
            else:
                #print("Contact Number : None")
                pass
            Contact_Number.append(mobile_number) 
            #print("\n")
            #print("----------------------------")
            
        next_disabled = soup.find_all('a',{'class':'dis','rel':True})
        if(next_disabled):
            for item in next_disabled:
                if(item['rel'][0] == 'next'):
                    Last_Page = True
                    break

        
        
         
    Content = pd.DataFrame({
                           'City': City,
                           'Property Type': Property,
                           'Property Name':Name,
                           'Contact Number': Contact_Number,
                           'Ratings': Rating
                           })
    #print(len(Property))
    driver.quit()
    return Last_Page,Content


In [98]:
City = 'Mumbai'
Property_type = 'PG'
Dataset = Generate_Data(City,Property_type)
Dataset

1
https://www.justdial.com/Mumbai/PG/page-1
2
https://www.justdial.com/Mumbai/PG/page-2


KeyboardInterrupt: 

In [99]:
! ls


export_dataframe_1_medium.csv  Mumbai_PG.csv
geckodriver.log		       Scraping Dummy IMDB.ipynb
Hostel_List.csv		       Scraping GoPGO.ipynb
movies.csv		       Scraping JustDial.ipynb
Mumbai_Hostel.csv	       Untitled.ipynb


In [100]:
Data = pd.read_csv('Mumbai_PG.csv')
Data

,Unnamed: 0,City,Property Type,Property Name,Contact Number,Ratings
0,0,Mumbai,PG,"Sahara Paying Guest in Kopar Khairane, Mumbai",+(91)-9152307574,4.0
1,1,Mumbai,PG,"Shree Krishna Paying Guest in Kopar Khairane, ...",+(91)-9152307770,3.9
2,2,Mumbai,PG,"Atharva Enterprises in Byculla, Mumbai",+(91)-9152377647,3.9
3,3,Mumbai,PG,"S L Villas in Marine Lines, Mumbai",+(91)-9152356517,3.8
4,4,Mumbai,PG,"Astha Property Solution in Kandivali East, Mumbai",+(91)-9152957380,3.8
...,...,...,...,...,...,...
95,95,Mumbai,PG,"Om Shiv Sai Real Estate Agency in Airoli, Mumbai",+(91)-8286061451,3.8
96,96,Mumbai,PG,"Vijay Enterprises in Airoli, Mumbai",+(91)-9152110640,4.5
97,97,Mumbai,PG,"Atithi Home Paying Guest in Airoli, Mumbai",+(91)-9223222261,4.3
98,98,Mumbai,PG,"Homelife Realty in Ic Colony-borivali West, Mu...",+(91)-9152894540,4.2
